# PreNet Training for Embedding Inversion
Here, we train our `PreNet` model for embedding to text inversion. We will use a sample of the `bookcorpus` dataset to train it.

In [1]:
import torch
from modules.inverter import PreNet, Inverter, get_encoder, get_gpt2_decoder
from modules.data import get_bookcorpus_for_inversion
from modules.train import train_inversion_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configs
We define the model and training hyperparameters.

In [29]:
models_config = {
    "encoder_id": "paraphrase-multilingual-MiniLM-L12-v2",
}

prenet_configs = {
    "input_dim": 384,
    "output_dim": 768,
    "rank": 128,
    "prefix_len": 20,
}

train_configs = {
    "load_weights": "saved_models/prenet_prefix_tuning_bookcorpus_multilingual.pth",
    "save_weights": "saved_models/prenet_prefix_tuning_bookcorpus_multilingual.pth",
    "lr": 1e-3,
    "max_target_len": 64,
    "embed_batch_size": 64,
    "train_batch_size": 64,
    "sample_data": 0.1,
    "num_epochs": 2,
}

---

## Models
We initialize and load the encoder, PreNet, and decoder mdoels.

In [17]:
encoder = get_encoder(models_config["encoder_id"])
decoder, tokenizer = get_gpt2_decoder()

prenet = PreNet(**prenet_configs).to(device)

if train_configs["load_weights"]:
    prenet.load_state_dict(
        torch.load(train_configs["load_weights"], map_location=device)
    )

## Dataset
We load the dataset using the `get_bookcorpus_for_inversion` function.

In [18]:
train_loader, val_loader = get_bookcorpus_for_inversion(
    encoder,
    tokenizer,
    train_configs["max_target_len"],
    train_configs["train_batch_size"],
    train_configs["embed_batch_size"],
    train_configs["sample_data"],
)

Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 105.30it/s]


## Training
Finally, we can train the model using the `train_inversion_model` function.

In [38]:
train_inversion_model(
    prenet,
    decoder,
    tokenizer,
    train_loader,
    val_loader,
    train_configs["lr"],
    train_configs["num_epochs"],
)

Epoch 1 [Valid]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:14<00:00, 10.77it/s]


Epoch 1/2 — Train Loss: 2.0928   Val Loss: 2.2391


Epoch 2 [Valid]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:14<00:00, 10.78it/s]

Epoch 2/2 — Train Loss: 2.0617   Val Loss: 2.2282


Save the updated weights of the model.

In [39]:
if train_configs["save_weights"]:
    torch.save(prenet.state_dict(), train_configs["save_weights"])

## Inference
Test the model at inference time using the `Inverter` class.

In [43]:
inverter = Inverter(prenet, decoder, tokenizer)

sample_text = "she was a physics student"
embedding = encoder.encode(sample_text, convert_to_tensor=True)

print(inverter.invert(embedding, max_len=30, temperature=0.1))

 she was a physics student . . . . . . . . . . . . . . . . . . . . . . . . .
